# Sistemes de Recomanació

En aquest segon lliurament es programarà un **sistema de recomanació**, que posarà en correspondència un *usuari* amb *ítems* en funció de les seves preferències i interessos. 
En aquesta ocasió, implementareu un sistema de recomanació que assisteixi en una compra.

## Abans de començar


**\+ Durant la pràctica, solament es podran fer servir les següents llibreries**:

`Pandas, Numpy, Itertools`

*Nota: A més de les que ja es troben presents en la 1a cel·la i funcions natives de Python*

**\+ No es poden modificar les definicions de les funcions donades, ni canviar els noms de les variables i paràmetres ja donats**

Això no implica però que els hàgiu de fer servir. És a dir, que la funció tingui un paràmetre anomenat `df` no implica que l'hàgiu de fer servir, si no ho trobeu convenient.

**\+ En les funcions, s'especifica que serà i de quin tipus cada un dels paràmetres, cal respectar-ho**

Per exemple (ho posarà en el pydoc de la funció), `df` sempre serà indicatiu del `Pandas.DataFrame` de les dades. Durant els testos, els paràmetres (i específicament `df`) no contindran les mateixes dades que en aquest notebook, si bé si seran del mateix tipus! Per tant, no us refieu de què tinguin, per exemple, el mateix nombre de files.

## Testos automàtics

Com ja sabeu, les pràctiques es fan a través de Github Classroom. Podeu treballar-hi lliurement i es recomana que feu commits sovint, per tal que els canvis quedin reflectits de forma estructurada i modular.

Normalment treballareu a la branca `master`, però podeu fer fins a 3 cops al dia un `commit` (o `merge` de `master`) a la branca `tests`. Això provocarà que es llencin un seguit de proves sobre el vostre codi, en podreu veure el resultat a la següent web: http://grade-me.education

Penseu que aquests testos són un subconjunt, petit, dels que realment farem servir per avaluar. Per tant, us recomanem que aprofiteu al màxim els 3 intents diaris, que us serviran per comprovar que els formats d'entrada i sortida siguin correctes, a més d'alguns testos bàsics de correcte funcionament.

# Preparar les dades

### **En aquesta cel·la no féu cap modificació**

Descomprimeix el zip en la carpeta "data" automàticament. La funció locate serveix per trobar la ruta relativa a la carpeta on s'està executant aquest python/ipython.

In [1]:
import zipfile
import pickle
from os.path import join, dirname


try:
    from IPython.core.display import HTML

    def pprint(df):
        with pd.option_context('display.max_rows', None, 'display.max_columns', None):
            display(HTML(pd.DataFrame(df).to_html()))
except:
    def pprint(df):
        pass

def locate(*path):
    base = globals().get('__file__', '.')
    return join(dirname(base), *path)

def unzip(file):
    zip_ref = zipfile.ZipFile(locate(file), 'r')
    zip_ref.extractall(locate('data'))
    zip_ref.close()

unzip('order_products__train.csv.zip')
unzip('orders.csv.zip')
unzip('products.csv.zip')

# Les dades

En aquest i futur notebooks farem servir dades reals corresponents a compres, concretament les utilitzades en el Kaggle Instacart Market Basket Analysis:
https://www.kaggle.com/c/instacart-market-basket-analysis


* **Order Products**: És el de major interès, conté la relació de productes comprats (`product_id`) per a cada conjunt de compra diferent (`order_id`). A aquests conjunts de compres ens hi referirem com a `ordres`, seguint la nomenclatura de les dades. A més, tot i que no ho farem servir, podríem arribar a saber en quin ordre s'han comprat els productes (`add_to_cart_order`) i inclús si ja s'havia comprat en alguna ordre anterior (`reordered`).

* **Orders**: Aquest dataset ens permet relacionar una compra en concret (`order_id`) amb l'usuari que l'ha feta (`user_id`)

* **Products**: Donat un `product_id` ens permet obtenir-ne més informació, com ara el nom (`product_name`), la secció en la qual es troba (`aisle_id`) o al departament al qual pertany (`department_id`). Aquests dos últims es complementen amb els conjunts **Aisles** i **Departments**.


# Data loading

### **En aquestes cel·les no feu cap modificació**

Carrega les dades en un DataFrame Pandas.

In [2]:
import pandas as pd

if __name__ == '__main__':
    df_order_prods = pd.read_csv(locate('data', 'order_products__train.csv'))
    df_orders = pd.read_csv(locate('data', 'orders.csv'))[['order_id', 'user_id']]
    df_prods = pd.read_csv(locate('data', 'products.csv'))[['product_id', 'aisle_id']]

# Implementació

Recordeu, seguiu els pydoc i compliu amb el que diuen!

El primer que haurem de fer és construir una matriu que ens serveixi, d'alguna forma, com a indicatiu de preferències de cada persona. Per tal efecte, construirem una matriu $m\times n$, de $m$ usuaris per $n$ items, on cada entrada $i,j$ serà el nombre de vegades que la persona $i$ a comprat l'item $j$.

<img src="img/Mat.png">

Per saber de quin usuari és cada `order_id`, haureu de creuar el dataset `order_products` amb el `orders`. Una sola persona/usuari tindrà més d'una ordre, mireu quants cops ha comprat els mateixos productes.

A més, les dades es componen de molts `product_id diferents`, hi ha massa diversitat entre usuaris. Per tant, per poder recomanar el que farem serà agregar les dades, en lloc de treballar per `product_id` ho farem per `aisle_id`, és a dir "la secció" del súper on es troba.

Al llarg de la pràctica es parlarà de producte i/o item, perquè és la terminologia estàndard de recomanadors, però sempre serà en referència a `aisle_id` per aquesta pràctica!

In [3]:
def merge_information(df_order_prods, df_orders, df_prods):
    """
    Retorna el dataframe resultant de:
        1. Creuar els datasets 'order_products' amb 'orders'.
        2. Creuar el dataframe anterior amb 'products'.
        Per creuar dos dataframes podeu utilitzar la funció pandas.DataFrame.merge

    :param df_order_prods: DataFrame 'order_products'
    :param df_orders: DataFrame 'orders'
    :param df_prods: DataFrame 'products'
    :return: DataFrame descrit prèviament   
    """
    
    # Creua df_order_prods i df_orders
    order_user = pd.merge(df_order_prods, df_orders, on='order_id')
    # Creua l'anterior amb df_products
    order_user_prod = pd.merge(order_user, df_prods, on='product_id')
    return order_user_prod

<div class='alert alert-info'>
Correcte
</div>

<div class='alert alert-warning'>
1/1
</div>

In [4]:
if __name__ == '__main__':
    df_merged = merge_information(df_order_prods, df_orders, df_prods)
    pprint(df_merged.head())

,order_id,product_id,add_to_cart_order,reordered,user_id,aisle_id
0,1,49302,1,1,112108,120
1,816049,49302,7,1,47901,120
2,1242203,49302,1,1,2993,120
3,1383349,49302,11,1,41425,120
4,1787378,49302,8,0,187205,120


In [5]:
def build_counts_table(df):
    """
    Retorna un dataframe on les columnes són els `aisle_id`, les files `user_id` i els valors
    el nombre de vegades que un usuari ha comprat un producte d'un `aisle_id`
    
    :param df: DataFrame original després de creuar-lo
    :return: DataFrame descrit adalt
    """
    # Usar el groupby i després separar possant els np.Nan a 0
    return df.groupby(['user_id','aisle_id'], sort=False).size().unstack(fill_value=0)

    # Un altra manera de fer-ho amb pivot:
    
    #df = df.groupby(['user_id','aisle_id'], sort=False).size()
    #df = df.reset_index()
    #df.columns = ['user_id', 'aisle_id','amount']
    #df = df.pivot(index='user_id', columns='aisle_id', values='amount')
    #df = df.fillna(0)

def get_count(df, user_id, aisle_id):
    """
    Retorna el nombre de vegades que un usuari ha comprat en un `aisle_id`
    
    :param df: DataFrame retornat per `build_counts_table`
    :param user_id: ID de l'usuari
    :param aisle_id: ID de la secció
    :return: Enter amb el nombre de vegades que ha comprat
    """
    return df.loc[user_id, aisle_id]

<div class='alert alert-info'>
Correcte
</div>

<div class='alert alert-warning'>
1/1
</div>

In [6]:
if __name__ == '__main__':
    df_counts = build_counts_table(df_merged)
    count = get_count(df_counts, 14, 5)
    print(count)

2


Tenim moltes dades en el nostre dataset, pel que és convenient que les reduïm una mica. Per començar a treballar recomanem que reduïu la mida a aproximadament 0.01 de l'original (`FRAC = 0.01`), mentre que per fer el Kaggle podeu utilitzar `FRAC=0.1` sense fer canvis, valors més alts requeriran més feina.

In [8]:
if __name__ == '__main__':
    FRAC = 0.001
    df_reduced = df_counts.sample(frac=FRAC, random_state=1)
    # Aqui contem quants items ha comprat cada usuari i filtrem -> Això es pel Kaggle ho deixem comentat
    #df_reduced = df_reduced[df_reduced.astype(bool).sum(axis=1) > 20] 
    pprint(df_reduced.head())
    print(df_reduced.shape)
    

aisle_id,120,108,83,95,24,21,2,115,53,123,86,96,117,19,116,16,31,71,81,91,78,106,129,36,67,59,37,74,84,99,34,17,75,69,72,98,128,25,111,85,54,107,9,133,11,112,63,88,23,70,82,100,6,87,32,45,48,114,104,56,12,52,30,51,66,110,4,3,130,7,77,61,79,94,64,13,35,125,43,1,105,80,131,38,121,93,62,26,50,20,15,68,49,127,14,40,8,92,29,122,60,10,22,27,118,47,57,65,55,89,76,42,124,119,46,41,73,134,109,58,28,5,126,97,39,103,90,33,101,132,18,44,102,113
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
156293,0,0,1,0,2,1,0,0,0,0,1,2,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
132307,0,0,0,0,2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
42533,1,0,0,0,4,0,0,2,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
172619,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
168574,0,1,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


(131, 134)


### Mesurament de similituds

El primer pas per poder recomanar és definir una funció de similitud entre vectors. Siguin $x, y$ vectors, de les següents propostes implementa'n mínim una:

* Distància euclidea (inversa): https://en.wikipedia.org/wiki/Euclidean_distance

$$sim(x, y) = \frac{1}{1 + \sqrt{\sum_i(x_i-y_i)^2}}\in [0, 1]$$

* Similitud cosinus: https://en.wikipedia.org/wiki/Cosine_similarity

$$sim(x, y) = \frac{x\cdot y}{||x||\hspace{0.1cm} ||y||} \in [-1,1]$$

* Correlació de Pearson: https://en.wikipedia.org/wiki/Pearson_correlation_coefficient

$${\displaystyle sim(x,y)={\frac {\sum _{i=1}^{n}(x_{i}-{\bar {x}})(y_{i}-{\bar {y}})}{{\sqrt {\sum _{i=1}^{n}(x_{i}-{\bar {x}})^{2}}}{\sqrt {\sum _{i=1}^{n}(y_{i}-{\bar {y}})^{2}}}}}} \in [-1,1] \\ \text{On }\bar{x} = \frac{1}{n} \sum^n_i x_i\text{ la mitja (i anàlogament per y)}$$

Per implementar qualsevol d'aquestes únicament es permet l'ús de:

* `np.sum`
* `np.sqrt`
* `np.power`
* `np.dot`
* `np.linalg.norm`
* `np.mean`

I s'ha de fer **sense bucles**.

<hr>

Tingueu en compte que les dues últimes funcions consideren valors negatius per exemples oposats (a diferència de la distància euclidea). En cas de fer servir alguna d'aquestes dues, pensa (més endavant) en com afectaran els negatius en la recomanació.

En la similitud cosinus, vigileu amb casos on un usuari no ha comprat res, tindreu a ser una divisió entre 0.

En la correlació de Pearson, haureu de considerar casos on algun dels dos exemples tingui variància 0, ja que aleshores estareu fent una divisió entre 0. En tals casos, podeu retornar un valor per defecte o alguna de les altres mesures.

In [9]:
import numpy as np

def similarity(x, y):
    """
    Definir quina de les similituds vols utilitzar  a l'execució.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la similitud
    """
    return cosine(x, y)

def euclid(x, y):
    """
    Retorna la distància euclidiana inversa de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la distància euclidiana
    """
    
    return 1/(1 + np.linalg.norm(x-y))

def cosine(x, y):
    """
    Retorna la similitud cosinus de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la similitud cosinus
    """
    
    # Cal comprobar que no siguin 0 per evitar errors
    if all(x == 0) or all(y == 0):
        return 0
    
    return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))

def pearson(x, y):
    """
    Retorna la correlació de Pearson de dos vectors n-dimensionals.
    
    :param x: Primer vector
    :param y: Segon vector
    :return : Escalar (float) corresponent a la correlació de Pearson
    """
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    num = np.sum(np.dot(x-mean_x, y-mean_y))
    var_x = np.sqrt(np.sum(np.power((x-mean_x),2)))
    var_y = np.sqrt(np.sum(np.power((y-mean_y),2)))
    
    # Si el denominador es 0 llavors return 0 per evitar error
    if var_x == 0.0 or var_y == 0.0:
        return 0.
    
    return num/(var_x*var_y)

<div class='alert alert-info'>
Correcte
</div>

<div class='alert alert-warning'>
1/1
</div>

In [10]:
if __name__ == '__main__':
    print(similarity(np.asarray([1, 1, 1]), np.asarray([1, 2, 3])))

0.925820099773


### Matriu de similituds

Per fer recomanació col·laborativa existeixen dues opcions, fer un recomanador basat en usuaris o un en ítems:

* Recomanador basat en usuaris:
Considera la matriu $M\times N: \text{usuaris}\times\text{items}$, per recomanar t'hauràs de basar en les similituds entre els usuaris.

* Recomanador basat en items:
Considera la matriu $M\times N: \text{items}\times\text{usuaris}$, per recomanar t'hauràs de basar en les similituds entre els ítems.

Construeix una matriu de mida $M\times M$ on cada posició $i,j$ indica la distància entre l'element $i$ i el $j$. Així doncs, si estàs fent un recomanador basat en usuaris, `matriu[2, 3]` contindrà la similitud entre l'usuari 2 i el 3. En canvi, si l'estàs fent basat en ítems, `matriu[2, 3]` contindrà la similitud entre l'ítem 2 i el 3.

Per simplificar els càlculs més endavant, la similitud d'un usuari (resp. ítem) amb sí mateix hauria de ser 0.

In [12]:
from tqdm import tqdm_notebook

def generate_data(similarity_function, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param similarity_function: Funció que calcularà la similitud 
        entre usuaris (resp. ítems)
    :param df_counts: Dataframe que conté el nombre de vegades que 
        un usuari ha comprat en un `aisle_id`
    :return simils: Matriu numpy de mida M x M amb les similituds.
    """
    # Hagafem els usuaris
    user_buys = df_counts.values.tolist()
    m = len(user_buys)
    # Creem una matriu M x M segons la quantitat d'usuaris que tenim (buida)
    simils = np.empty((m,m))
    
    # Si tqdm dóna error llavors cal usar -> for i in range(m):
    for i in tqdm_notebook(range(m)):
        first_user = user_buys[i]
        for j in range(len(user_buys)):
            if i < j: # L'index es troba en la matriu superior -> fer el càlcul de similitud
                second_user = user_buys[j]
                simil = similarity_function(np.asarray(first_user), np.asarray(second_user))
                simils[i][j] = float("{0:.5f}".format(simil))
            elif i == j: # Quan es diagonal ficar un 0
                simils[i][i] = 0.
            else: # Si es la matriu inferior llavors copiem el valor ja calculat
                simils[i][j] = simils[j][i]
    return simils


In [13]:
def similarity_matrix(similarity_function, df_counts):
    """
    Retorna una matriu de mida M x M on cada posició 
    indica la similitud entre usuaris (resp. ítems).
    
    :param similarity_function: Funció que calcularà la similitud 
        entre usuaris (resp. ítems)
    :param df_counts: Dataframe que conté el nombre de vegades que 
        un usuari ha comprat en un `aisle_id`
    :return : Matriu numpy de mida M x M amb les similituds.
    """
    
    # Cal comprobar si la funció es None o no per tal de treballar amb funció matricial o
    # amb les funcions que ja hem implementat abans per fer-ho amb vectors
    
    if similarity_function is None: # Faster using matricial function
        
        # Base similarity matrix (all dot products)
        # Replace this with A.dot(A.T).toarray() for sparse representation
        base_matrix = df_counts.values
        base_matrix_prod = np.dot(base_matrix, base_matrix.T)

        # Squared magnitude of preference vectors (number of occurrences)
        square_mag = np.diag(base_matrix_prod)

        # Inverse squared magnitude
        inv_square_mag = 1 / square_mag

        # if it doesn't occur, set it's inverse magnitude to zero (instead of inf)
        inv_square_mag[np.isinf(inv_square_mag)] = 0

        # inverse of the magnitude
        inv_mag = np.sqrt(inv_square_mag)

        # Cosine similarity (elementwise multiply by inverse magnitudes)
        cosine = base_matrix_prod * inv_mag
        cosine = cosine.T * inv_mag
        
        # Fill diagonal values with 0
        np.fill_diagonal(cosine, 0)
        
        return cosine
    
    else:        
        # Generate the Data for numpy array
        return generate_data(similarity_function, df_counts)

<div class='alert alert-info'>
Molt bé totes dues implementacions, tant la iterativa on teniu en compte que la matriu és simètrica i per tant únicament cal calcular una triangular, com la matricial
</div>

<div class='alert alert-warning'>
4/4
</div>

Per cridar aquesta funció, el primer paràmetre pot ser:

* Alguna de les funcions que has programat abans (*euclid*, *cosine* o *pearson*) (~@2h treballant directament amb valors de numpy, ~@20h a partir de pandas pur)
* Opcionalment (no és obligatori fer-ho) podeu programar una funció que treballi específicament amb matrius (i no vectors). Si ho feu, cal gestionar-ho quan es rep `None`. No totes les funcions anteriorment anomenades són fàcils (ni intuïtives, ni hi caben en memòria) d'aplicar en forma matricial.  (@5s)

In [14]:
if __name__ == '__main__':
    try: 
        with open('similarities.pkl', 'rb') as fp:
            similarities = pickle.load(fp)
    except:
        similarities = similarity_matrix(None, df_reduced) # None, df_reduced : for cos or similarity
        with open('similarities.pkl', 'wb') as fp:
            pickle.dump(similarities, fp, pickle.HIGHEST_PROTOCOL)

### Generació de prediccions

Per fer recomanació col·laborativa, necessitem una funció que ens doni un valor de quant bona seria la recomanació. En el nostre cas i amb les nostres dades, volem una funció que ens indiqui quants cops compraria un usuari un producte donat.

* Si esteu fent un recomanador basat en usuaris, la puntuació per a un usuari $u$ i ítem $j$ és

$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{p\neq u,r_{p,i}>0} sim(u, p)\cdot r_{p,i}}{\sum_{p\neq u,r_{p,i}>0} sim(u, p)}$$

On $r_{u,i}$ indica el nombre de vegades que l'usuari $u$ ha comprat l'l'ítem $i$.

És a dir, per cada usuari $p$ diferent de $u$ si aquest usuari ha comprat algun cop el producte $i$, la similitud entre $p$ i $u$ multiplicada pel nombre de vegades que l'usuari $p$ ha comprat l'l'ítem $i$ ($r_{p,i}$).

Pondera't per la suma de les similituds.

* Anàlogament, si està basat en ítem, la puntuació per a un usuari $u$ i ítem $j$ és

$$pred(u, i) = \hat{r}_{u,i} = \frac{\sum_{j\neq i,r_{u,j}>0} sim(i, j)\cdot r_{u,j}}{\sum_{j\neq i,r_{u,j}>0} sim(i, j)}$$

On $r_{u,i}$ indica el nombre de vegades que l'usuari $u$ ha comprat l'ítem $j$.

És a dir, per cada ítem $j$ diferent de $i$ si l'usuari al qui recomanem ha comprat l'ítem $j$, la similitud entre $i$ i $j$ multiplicada pel nombre de vegades que l'usuari al qui recomanem $u$ ha comprat l'ítem $j$ ($r_{u,j}$)

Pondera't per la suma de les similituds.

Fixeu-vos que, sigui quin sigui el cas, al final estem fent el producte vectorial entre dos vectors. Concretament, el producte vectorial entre les similituds i les compres. Fes una funció que calculi aquest resultat:

In [15]:
def calc_score(sims, counts, user_mean, means):
    """
    * Si estàs implementant basat en usuaris:
        Donades les similituds i el nombre de vegades que l'usuari ha comprat
        cada ítem, retorna la predicció que indica quants cops compraria 
        l'usuari un nou ítem.
    
        :param sims: Similituds entre usuaris
        :param counts: Nombre de vegades que l'usuari ha comprat cada ítem,
        :return : Predicció (float) que indica quants cops compraria l'usuari un ítem.
        
        
    * Si estàs implementant basat en items:
        Donades les similituds i el nombre de vegades que l'item a recomanar ha
        estat comprat per cada usuari, retorna la predicció que indica quants cops
        compraria l'usuari un nou ítem.
    
        :param sims: Similituds entre itemsc
        :param counts: Nombre de vegades que l'ítem ha estat comprat per cada usuari
        :return : Predicció (float) que indica quants cops compraria l'usuari un ítem.
    """
    
    # Primer restem la quantitat de productes que ha comprat un usuari respecte la seva mitja
    norms = np.subtract(counts, means)

    # Calculem el numerador multiplicant per el vector de similituds
    # Aixo no deixa de ser una multiplicació de vectors ja que:
    # Per exemple: sims = [0.0, 0.6, 0.5, 0.35] & counts=[0,1,3,0] 
    # Els que no han comprat res dona 0/k = 0 i pel mateix usuari la similitud ja és 0 i ha comprat 0 cops
    numerador = np.dot(sims, norms)

    # Nomes sumar els usuaris que han comprat
    idx_of_buyers_to_sum = np.where(counts != 0)[0]
    denominador = sims[idx_of_buyers_to_sum].sum()
    
    return user_mean + (numerador/denominador)

<div class='alert alert-info'>
En general està bé, però compte amb el numerador. Només s'ha de tenir en compte aquelles persones que han comprat els items. Això normalment ja passa de per sí, doncs counts és 0 per aquests i per tant la multiplicació dóna 0. Ara bé, al restar la mitja és una història diferent. Es podria fer algo tipus `numerador = np.dot(sims[counts > 0], norms[counts > 0])`.
</div>

<div class='alert alert-warning'>
2/3
</div>

In [17]:
def score(user, item, df, similarities):
    """
    Extreu les similituds i el nombre de vegades que un usuari ha comprat un ítem
    (resp. nombre de vegades que cada usuari ha comprat l'ítem) i crida a la funció 
    anterior per calcular les prediccions.
    
    :param user: ID de l'usuari per la predicció
    :param item: ID de l'ítem per la predicció
    :param df: Dataframe que conté el nombre de vegades que un usuari 
        ha comprat en un `aisle_id`
    :param similarities: Matriu de similituds
    :return : Retorna un escalar (float) amb la predicció
    
    """
    # He decidit implementar aquesta funció amb la millora que es recomana més abaix

    # Calcular la mitjana i hagafar la de l'usuari en qüestió
    means = df.mean(axis=1)
    user_buy_mean = means.loc[user]
    
    # Items que ha comprat cada usuari
    users_buy_vect = np.asarray(df[item].values)
    
    # Vector de similituds de l'usuari en qüestió respecte la resta d'usuaris
    idx_of_user = df.index.get_loc(user)
    users_sim_vect = similarities[idx_of_user]
    
    return calc_score(users_sim_vect, users_buy_vect, user_buy_mean, means)

<div class='alert alert-info'>
Correcte
</div>

<div class='alert alert-warning'>
1/1
</div>

In [18]:
if __name__ == '__main__':
    print(score(df_reduced.index[0], df_reduced.columns[0], df_reduced, similarities))

1.2887227959


Feu una funció que donat un usuari calculi per cada item que no ha comprat la puntuació d'aquest. La funció retorna els $N$ items més ben puntuats.

In [19]:
from queue import PriorityQueue

def recommend_n_items(user_id, df, similarities, N):
    """
    Donat un usuari calcula per cada ítem que no ha comprat la puntuació d'aquest. 
    La funció retorna els $N$ ítems més ben puntuats.
    
    :param user_id: Identificador de l'usuari
    :parma df: Dataframe que conté el nombre de vegades que un usuari 
        ha comprat en un `aisle_id`
    :param similarities: Matriu de similituds
    :param N: Nombre d'ítems que volem que siguin recomanats.
    :return : Llista amb els IDs dels ítems recomanats
    """
    # Creem una cua de prioritats
    q = PriorityQueue() 
    # Hagafem les localitzacions dels items no comprats que formen part dels possibles items a recomanar
    no_comprats = df.loc[user_id] == 0 
    posibles_recom = df.loc[user_id].loc[no_comprats].index.tolist()
    for item in posibles_recom:
        # Calculem l'score per aquest possible item i el fiquem a la cua
        valor = score(user_id, item, df, similarities)
        q.put((-valor, item))
      
    recomanacio = []
    prod = []
    i = 0
    # Hagafem els N items que volem recomanar amb un millor valor
    while not q.empty() and i < N:
        next_item = q.get()
        recomanacio.append((abs(next_item[0]), next_item[1]))
        prod.append(next_item[1])
        i += 1
    return prod
        
    


<div class='alert alert-info'>
Correcte
</div>

<div class='alert alert-warning'>
1/1
</div>

In [204]:
if __name__ == '__main__':
    print(recommend_n_items(df_reduced.index[0], df_reduced, similarities, 10))

[123, 120, 107, 116, 115, 37, 112, 31, 16, 78]


<div class='alert alert-success'>
11/12
</div>

#### Possibles millores 

** 0) Utilització completa de les dades:**
Fer servir `df_original` tindrà (possiblement) resultats més fiables, però trigarà molt més que amb la versió reduida `df`. Pots canviar el `FRAC` a valors més alts ($\leq 1$) per utiltizar més dades, però compte perque potser la matriu $M\times M$ no hi cabrà en memòria.

**1) Normalització: Prediccions escalades al domini de l'usuari:**
Els usuaris compren en diferent mesura els productes, un més quantitat, altres menys. Fent servir la següent formula, escalem la predicció  a la mitja de l'usuari:
$$pred(u, i) = \hat{r}_{u,i} = \bar{r_u} + \frac{\sum_{p\neq u,r_{p,i}>0} sim(u, p)\cdot (r_{p,i}-\bar{r_b})}{\sum_{p\neq u,r_{p,i}>0} sim(u, p)}$$
on $\bar{r_u}$ és la mitjana de compres de l'usuari *u*.
    
**2) Valor del nombre d'elements codificats: **
Redueix la similitud entre els usuaris si el nombre de productes és baix o descarta (en entrenament) aquells usuaris amb un petit nombre de productes comprats.

**3) Augment de la similitud: **
Incrementeu el pes als usuaris que són realment similars (~ = 1)

**4) Selecció d'usuaris semblants: **
Només s'utilitza un subconjunt d'usuaris similars, descartant tots aquells usuaris poc similars.


Totes aquestes tècniques es poden aplicar d'igual manera en la recomanació col·laborativa per usuaris o ítems.

# Kaggle

Per als usuaris que tens a continuació, quins productes els hi recomanaries (**fins a un màxim de 5**) que compressin segons el que ja han comprat?

https://www.kaggle.com/t/d7a790f107ca43aca34a12bace9985cd

In [187]:
if __name__ == '__main__':
    df_test_products = pd.read_csv(locate('order_products__test.csv'))
    df_test_orders = pd.read_csv(locate('orders__test.csv'))[['order_id', 'user_id']]
    df_test_merged = merge_information(df_test_products, df_test_orders, df_prods)
    
    df_test_counts = build_counts_table(df_test_merged)
    df_all = df_reduced.append(df_test_counts)
    df_all = df_all.fillna(0)
    print(df_all.tail())

aisle_id  1    2    3    4    5    6    7    8    9    10  ...   125  126  \
user_id                                                    ...              
300033      2    0   17    0    0    0    0    0    0    0 ...     0    0   
300046      0    0    0   26   26    0    0    0    0    0 ...     0    0   
300142      0    0   15    0   15   18   15   18   18    0 ...     0    0   
300014      0    0    0    5    0    0    0    0    0    0 ...     0    0   
300112      0    0    0    5    0   19   19    0    0    0 ...     0    0   

aisle_id  127  128  129  130  131  132  133  134  
user_id                                           
300033      0    0    0   10   10    0    0    0  
300046      0    0    0   26   26    0    0    0  
300142      0    0    0    0   13    0    0    0  
300014      0    9    9   24    0    0    0    0  
300112      0    8   11    0   19    0    0    0  

[5 rows x 134 columns]


In [188]:
if __name__ == '__main__':
    try: 
        with open('similarities_test.pkl', 'rb') as fp:
            similarities_test = pickle.load(fp)
    except:
        similarities_test = similarity_matrix(None, df_all) # similarity
        with open('similarities_test.pkl', 'wb') as fp:
            pickle.dump(similarities_test, fp, pickle.HIGHEST_PROTOCOL)

In [189]:
if __name__ == '__main__':
    df_submission = pd.DataFrame(columns=['user_id', 'products_list'])

    for user_id in df_test_counts.index:
        user_recos = recommend_n_items(user_id, df_all, similarities_test, 5)

        df_submission = df_submission.append(
            {
                'user_id': user_id,
                'products_list': ' '.join(map(str, user_recos))
            }, 
            ignore_index=True)

    df_submission.to_csv('./susbmission.csv', index=None)

In [190]:
if __name__ == '__main__':
    print(df_submission.head())

  user_id      products_list
0  300097   123 83 96 93 106
1  300130   123 21 96 108 91
2  300111     21 96 31 93 91
3  300007    84 115 91 59 37
4  300107  123 112 96 21 115
